In [1]:
# Homework 6 Template
# G. Besla & R. Li

In [2]:
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G
from pathlib import Path

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [3]:
# my modules
from galaxy.galaxy import Galaxy
# Step 1: modify CenterOfMass so that COM_P now takes a parameter specifying 
# by how much to decrease RMAX instead of a factor of 2
from galaxy.centerofmass import CenterOfMass

Start by finding out what we're dealing with in terms of particle numbers:

In [4]:
from galaxy.galaxies import Galaxies

g_small = Galaxies(datadir='./VLowRes')
g_small.get_counts_pivot()

typename,1 Halo,2 Disk,3 Bulge,All
name,,,,
M31,25000,60000,9500,94500
M33,5000,9300,0,14300
MW,25000,37500,5000,67500
All,55000,106800,14500,176300


In [52]:
def orbit_com(galname, start, end, n, datadir='.'):
    """
    function that loops over all the desired snapshots to compute the COM pos and vel as a function of time.
    inputs:
          
    returns: 
    """
    
    # compose the filename for output
    fileout = f'./orbit_{galname}.txt'
    
    #  set tolerance and vol_dec for calculating COM_P in CenterOfMass
    # for M33 that is stripped more, use different values for vol_dec
    if galname == 'M33':
        delta = 0.1
        vol_dec = 4
    else:
        delta = 0.2
        vol_dec = 2
    
    # generate the snapshot id sequence 
    # it is always a good idea to also check if the input is eligible (not required)
    snap_ids = np.arange(start, end, n)
    
    # initialize the array for orbital info: t, x, y, z, vx, vy, vz of COM
    orbit = np.zeros((len(snap_ids), 7))
    
    print(galname)
    
    for  i, snap in enumerate(snap_ids): # loop over files
        gal = Galaxy(galname, snap, datadir=datadir)
        
        # Initialize an instance of CenterOfMass class, using disk particles
        com = CenterOfMass(gal)

        # Store the COM pos and vel. Remember that now COM_P required VolDec
        com_p = com.com_p(delta, vol_dec)
        com_v = com.com_v(com_p)
    
        # store the time, pos, vel in ith element of the orbit array,  without units (.value) 
        # note that you can store 
        # a[i] = var1, *tuple(array1)
        orbit[i] = gal.time.value, *tuple(com_p.value), *tuple(com_v.value)
#         orbit[i] = gal.time, *tuple(com_p).value, *tuple(com_v).value

        
        # print snap_id to see the progress
        print(snap, end=' ')
    
    print('\nDone')
        
    # write the data to a file
    # we do this because we don't want to have to repeat this process 
    # this code should only have to be called once per galaxy.
    np.savetxt(fileout, orbit, fmt = "%11.3f"*7, comments='#',
               header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                      .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))

In [55]:
# Recover the orbits and generate the COM files for each galaxy
# read in 800 snapshots in intervals of n=5
# Note: This might take a little while - test your code with a smaller number of snapshots first! 

for galname in ('MW', 'M31', 'M33'):
    gal = Galaxy('M33', datadir='./VLowRes')
    com = CenterOfMass(gal)
    orbit_com(galname, 0, 800, 5, datadir='./VLowRes')

MW
0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180 185 190 195 200 205 210 215 220 225 230 235 240 245 250 255 260 265 270 275 280 285 290 295 300 305 310 315 320 325 330 335 340 345 350 355 360 365 370 375 380 385 390 395 400 405 410 415 420 425 430 435 440 445 450 455 460 465 470 475 480 485 490 495 500 505 510 515 520 525 530 535 540 545 550 555 560 565 570 575 580 585 590 595 600 605 610 615 620 625 630 635 640 645 650 655 660 665 670 675 680 685 690 695 700 705 710 715 720 725 730 735 740 745 750 755 760 765 770 775 780 785 790 795 
Done
M31
0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180 185 190 195 200 205 210 215 220 225 230 235 240 245 250 255 260 265 270 275 280 285 290 295 300 305 310 315 320 325 330 335 340 345 350 355 360 365 370 375 380 385 390 395 400 405 410 415 420 425 430 435 440 445 450 455 460 465 470 475 480 485

In [ ]:
# Read in the data files for the orbits of each galaxy that you just created
# headers:  t, x, y, z, vx, vy, vz
# using np.genfromtxt



In [ ]:
# function to compute the magnitude of the difference between two vectors 
# You can use this function to return both the relative position and relative velocity for two 
# galaxies over the entire orbit  


In [ ]:
# Determine the magnitude of the relative position and velocities 

# of MW and M31

# of M33 and M31


In [ ]:
# Plot the Orbit of the galaxies 
#################################



In [ ]:
# Plot the orbital velocities of the galaxies 
#################################
